# Welcome to ExKaldi

In this section, we will make a HCLG decode graph.

In [1]:
import os
dataDir = "librispeech_dummy"

os.environ["LD_LIBRARY_PATH"] = "/home/khanh/workspace/miniconda3/envs/kaldi/lib/;/home/khanh/workspace/miniconda3/envs/test/lib/"

import exkaldi
exkaldi.info.reset_kaldi_root("/home/khanh/workspace/projects/kaldi")

exkaldi.info.reset_kaldi_root( yourPath )
If not, ERROR will occur when implementing some core functions.


Prepare lexicons (generated in 3_prepare_lexicons).

In [2]:
lexFile = os.path.join(dataDir, "exp", "lexicons.lex")

lexicons = exkaldi.load_lex(lexFile)

Like the traing of HMM and decision tree, we also have high-level API to make HCLG graph. But firstly we introduce how to build HCLG graph with exkaldi toolkit in detail.

### Make HCLG in detail

In [3]:
outDir = os.path.join(dataDir, "exp", "train_delta", "graph")

exkaldi.utils.make_dependent_dirs(outDir, pathIsFile=False)

Then compose __L.fst__ and __G.fst__ to __LG.fst__. 

__L.fst__ file has been generated before (in 3_prepare_lexicons), use it directly.  
__G.fst__ file has been generated before (in 04_train_and_query_language_model), use it directly.

In [4]:
Lfile = os.path.join(dataDir, "exp", "L_disambig.fst")
Gfile = os.path.join(dataDir, "exp", "G.fst")
LGfile = os.path.join(outDir, "LG.fst")

exkaldi.decode.graph.compose_LG(Lfile, Gfile, outFile=LGfile)

'/home/khanh/workspace/projects/exkaldi/tutorials/librispeech_dummy/exp/train_delta/graph/LG.fst'

Then compose __LG__ and context information to __CLG.fst__. 

__tree__ will be used here. and __ilabel__ info will also be generated in this step.

In [5]:
treeFile = os.path.join(dataDir, "exp", "train_delta", "tree")
CLGfile = os.path.join(outDir, "CLG.fst")

_, ilabelFile = exkaldi.decode.graph.compose_CLG(lexicons, treeFile, LGfile, outFile=CLGfile)

ilabelFile

'librispeech_dummy/exp/train_delta/graph/CLG.ilabels'

Last, compose all infos to final __HCLG.fst__ graph.

In [6]:
hmmFile = os.path.join(dataDir, "exp", "train_delta", "final.mdl")
HCLGFile = os.path.join(outDir, "HCLG.fst")

exkaldi.decode.graph.compose_HCLG(hmmFile, treeFile, CLGfile, ilabelFile, outFile=HCLGFile)

'librispeech_dummy/exp/train_delta/graph/HCLG.fst'

### Make HCLG with high-level API

In [7]:
os.remove(LGfile)
os.remove(CLGfile)
os.remove(ilabelFile)
os.remove(HCLGFile)

In [8]:
exkaldi.decode.graph.make_graph(lexicons, hmmFile, treeFile, tempDir=outDir, useLFile=Lfile, useGFile=Gfile)

Skip making Lexicon fst. Use: librispeech_dummy/exp/L_disambig.fst.
Skip making Grammar. Use: librispeech_dummy/exp/G.fst.
Compose LG done: /home/khanh/workspace/projects/exkaldi/tutorials/librispeech_dummy/exp/train_delta/graph/LG.fst.
Compose CLG done: librispeech_dummy/exp/train_delta/graph/CLG.fst.
Ilabel info: librispeech_dummy/exp/train_delta/graph/CLG.ilabels.
Make HCLG done: librispeech_dummy/exp/train_delta/graph/HCLG.fst.


'librispeech_dummy/exp/train_delta/graph/HCLG.fst'

Here we specified "useLfile" and "useGfile" to use the L and G fst file generated before. If you don't want this, specify "useLFile" and "useGFile" None, and other parameters, such as "useDisambigLexicons" and so on.